In [1]:
import numpy as np
import pandas as pd
from tqdm import tqdm
pd.set_option('display.width', 100)
pd.set_option('display.max_columns', None)

import seaborn as sea
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go

from nanoporePipelineCommon import *

print("imports done")

imports done


### Loading files
First goal will be to load files in to the environment in a method amenable to retaining the most date, filtering can be done after the fact!

In [2]:
# reads_df_raw, compressed_df_raw = load_and_merge_lib_parquets(["xrn-1-5tera", "xrn-1-5tera-smg-6"],
#                                                               drop_sub_n=1, add_tail_groupings=False,
#                                                               group_by_t5=True)
reads_df_transcripts_raw, compressed_df_transcripts_raw = load_and_merge_lib_parquets(["xrn-1-5tera", "xrn-1-5tera-smg-6"],
                                                              drop_sub_n=1, add_tail_groupings=False,
                                                              group_by_t5=True, keep_transcript_info=True,
                                                              read_pos_in_groupby=True)
print("loading done")

Loading readAssignments file from: /data16/marcus/genomes/elegansRelease100/Caenorhabditis_elegans.WBcel235.100.allChrs.parquet... Done.
Looking for files for libraries: ['xrn-1-5tera', 'xrn-1-5tera-smg-6']
Looking for file for xrn-1-5tera, at /data16/marcus/working/211118_nanoporeRun_totalRNA_5108_xrn-1-KD_5TERA/output_dir/merge_files/*_mergedOnReads.parquet... File Found.
Looking for file for xrn-1-5tera-smg-6, at /data16/marcus/working/211210_nanoporeRun_totalRNA_2102_xrn-1-KD_5TERA/output_dir/merge_files/*_mergedOnReads.parquet... File Found.
Loading parquet for xrn-1-5tera lib... Done.
'original_chr_pos' column already found in dataframe, skipping adjustment for 5'ends!
Loading parquet for xrn-1-5tera-smg-6 lib... Done.
'original_chr_pos' column already found in dataframe, skipping adjustment for 5'ends!
Finished assignment merge!. . .

Read counts pre-failed-polyA call drop:   1658891
Read counts post-failed-polyA call drop:  1092362
Read counts post gene assignment:  1092362
Rea

Storing start distances as lists: 100%|██████████| 43829/43829 [00:00<00:00, 48841.91it/s]


Gene counts pre sub-1 transcript_hits drop:  43829
Gene counts post sub-1 transcript_hits drop:  43829
loading done


### Quickly copy dataframes...
... so we can revert to disk originals w/out having to reload from disk

In [3]:
reads_df_transcripts = reads_df_transcripts_raw.copy()
compressed_df_transcripts = compressed_df_transcripts_raw.copy()

### The one code block below only works for compressed_on_genes, not compressed_on_transcripts!!

In [142]:
# Below code only works when using the compressed_on_genes dataframes, it'll be broken for compressed on transcripts!!
reads_df_genes_raw, compressed_df_genes_raw = load_and_merge_lib_parquets(["xrn-1-5tera", "xrn-1-5tera-smg-6"],
                                                              drop_sub_n=1, add_tail_groupings=False,
                                                              group_by_t5=True)
reads_df_genes, compressed_df_genes = reads_df_genes_raw.copy(), compressed_df_genes_raw.copy()
drop_genes_with_less_than = 10
droped_sub_X_hits_df = compressed_df_genes.query(f"gene_hits > {drop_genes_with_less_than}")

adapted_compressed_df = droped_sub_X_hits_df.query("t5 == '+'")
unadapted_compressed_df = droped_sub_X_hits_df.query("t5 == '-'")

cols_for_merge = ["lib", "gene_id", "gene_name", "chr_id", "gene_hits","gene_rpm", "mean_polya_length"]
just_tails_merge = pd.merge(adapted_compressed_df[cols_for_merge],
                            unadapted_compressed_df[cols_for_merge],
                            on=["lib", "gene_id", "gene_name", "chr_id"], suffixes=("_adapted", "_unadapted"))
just_tails_merge['isMtDNA'] = just_tails_merge["chr_id"] == "MtDNA"
import plotly.io as pio
pio.renderers.default = "browser"
col_to_plot = "mean_polya_length"
fig = px.scatter(just_tails_merge.query("lib == 'xrn-1-5tera'"),
                 x=f"{col_to_plot}_adapted",
                 y=f"{col_to_plot}_unadapted",
                 trendline='ols',
                 hover_name="gene_name",
                 hover_data=["gene_id", "lib",
                             "gene_rpm_adapted",
                             "gene_rpm_unadapted",
                             "mean_polya_length_adapted",
                             "mean_polya_length_unadapted"],
                 log_x=True, log_y=True,
                 color="isMtDNA")
# fig = px.density_contour(just_tails_merge.query("lib == 'xrn-1-5tera'"),
#                          x=f"{col_to_plot}_adapted",
#                          y=f"{col_to_plot}_unadapted",
#                          # log_x=True, log_y=True,
#                          marginal_x='histogram', marginal_y='histogram')

# fig.update_layout(template='plotly_white')
fig

Loading readAssignments file from: /data16/marcus/genomes/elegansRelease100/Caenorhabditis_elegans.WBcel235.100.allChrs.parquet... Done.
Looking for files for libraries: ['xrn-1-5tera', 'xrn-1-5tera-smg-6']
Looking for file for xrn-1-5tera, at /data16/marcus/working/211118_nanoporeRun_totalRNA_5108_xrn-1-KD_5TERA/output_dir/merge_files/*_mergedOnReads.parquet... File Found.
Looking for file for xrn-1-5tera-smg-6, at /data16/marcus/working/211210_nanoporeRun_totalRNA_2102_xrn-1-KD_5TERA/output_dir/merge_files/*_mergedOnReads.parquet... File Found.
Loading parquet for xrn-1-5tera lib... Done.
'original_chr_pos' column already found in dataframe, skipping adjustment for 5'ends!
Loading parquet for xrn-1-5tera-smg-6 lib... Done.
'original_chr_pos' column already found in dataframe, skipping adjustment for 5'ends!
Finished assignment merge!. . .

Read counts pre-failed-polyA call drop:   937835
Read counts post-failed-polyA call drop:  607422
Read counts post gene assignment:  607422
Read c

Counting reads per gene: 100%|██████████| 22466/22466 [00:00<00:00, 65419.11it/s]


Gene counts pre sub-1 gene_hits drop:  22466
Gene counts post sub-1 gene_hits drop:  22466


### New columns
Create new columns to retain info regarding:
 - If a read made it out to the 5' UTR

In [5]:
reads_df_transcripts["spanningCDS"] = reads_df_transcripts["to_start"] > 0
wt_reads_df = reads_df_transcripts.query("lib == 'xrn-1-5tera'")
wt_reads_df

,lib,read_id,chr_id,chr_pos,gene_id,gene_name,cigar,sequence,polya_length,strand,transcript_id,to_start,to_stop,t5,read_length,spanningCDS
0,xrn-1-5tera,d85a9afc-c20f-44b7-b5b9-78cc98562cff,I,10227,WBGene00022277,homt-1,5S24M1D4M2D19M3D2M1I34M1I10M1D18M2D30M1D22M1D1...,GCTTTTAAAATATTGCTTAACATTTATTTACAGATAACGTGAGCAC...,131.91,-,Y74C9A.3.1,-79,-780,-,873,False
1,xrn-1-5tera,ccc75695-0f7c-4536-9721-681d57d9bfc6,I,10229,WBGene00022277,homt-1,11S26M1D20M3D5M1D37M1D53M1D25M1I21M1I23M1I4M1I...,ATTTTTTTTTTAATATTGCTTAACATTTATTTAACAGAATAACGTG...,67.45,-,Y74C9A.3.1,-81,-782,-,897,False
2,xrn-1-5tera,a8d4dad6-30a7-4dec-8fe1-9bfaba2477be,I,10224,WBGene00022277,homt-1,93S11M4D32M3D20M1I13M1D2M1D7M2D82M1D11M1D4M1I3...,GTTTGAGGTGTTGAAATGTGGGAGGGGATAGGTTGTAAATGTATAT...,83.21,-,Y74C9A.3.1,-76,-777,-,969,False
3,xrn-1-5tera,6fc1d960-5642-416b-9a96-b5450c824f6f,I,6097,WBGene00022277,homt-1,91S11M4D11M1D7M1D9M2D8M1D17M3D41M1D48M2D3M1D22...,GTTTATGTAAAGGGAATATGAGGTATGGGGATAGGGTGATGTAATG...,22.00,-,Y74C9A.3.1,404,-297,-,470,True
4,xrn-1-5tera,79a3bc2e-17b6-4691-ba65-cd22d69d3ec2,I,9840,WBGene00022277,homt-1,9S27M1I19M4D7M1I64M2I37M1D33M2D21M3D22M836N4M1...,AGTTTTTTTATATTGCTTAACATTTATTTAACAGAAGATAACGTGA...,60.46,-,Y74C9A.3.1,60,-641,-,750,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
783969,xrn-1-5tera,e0b97c3e-1b99-4529-b4e3-40a6521652bf,X,17717406,WBGene00007068,cTel55X.1,7S38M991N6M2D1M1I14M1D6M1D1M1D39M2I3M1I2M4I20M...,ACACCATAAGGGAATCGCTGTTTTTGTGACGCTCTACTTGATTGAC...,19.04,+,cTel55X.1b.1,444,-254,-,332,True
783970,xrn-1-5tera,6510a264-686e-4e28-bbf5-d6b3333455d8,X,17718435,WBGene00007068,cTel55X.1,62S14M2D32M1I1M2I19M1I4M1I25M1D3M1I6M2D2M1I15M...,ATACGTTTTCAACCGGTGCCATTCCAAGGATCGCTGTCTTTGTGAC...,79.08,+,cTel55X.1a.1,482,-183,-,339,True
783971,xrn-1-5tera,6510a264-686e-4e28-bbf5-d6b3333455d8,X,17718435,WBGene00007068,cTel55X.1,62S14M2D32M1I1M2I19M1I4M1I25M1D3M1I6M2D2M1I15M...,ATACGTTTTCAACCGGTGCCATTCCAAGGATCGCTGTCTTTGTGAC...,79.08,+,cTel55X.1b.1,489,-209,-,339,True
783972,xrn-1-5tera,82b9ec06-1374-4467-9195-98066b6a7792,X,17718548,WBGene00007068,cTel55X.1,5S13M7D3M1I36M1D22M1D23M1D8M5D50M62S,TCCTAGCTACGTGTGGCTCTCAAGTGCCCTTGTGCACGGAGTCGAG...,25.36,+,cTel55X.1a.1,595,-70,-,223,True


### Custom compress on transcripts
~~~While the original ***load_and_merge_lib_parquets*** method already spit out the ***compressed_df*** that has most of the ***read_df*** information organized based on *transcripts*, it didn't retain information about if reads spanned the CDS. So now we'll mostly recreate that, with the additional information regarding CDS spanning reads, like we want to plot by!!~~~

lol nvm... It looks like by using ***read_pos_in_groupby=True***, we are able to retain the start/stop information without rewriting the whole compress script. Thank you past me!!

We'll still need to convert that list of ***to_start*** distances to a fraction of reads spanning the CDS. This should be easy w/ a quick apply method that will take care of turning the lists into fractions!

In [6]:
def list_to_fraction_spanning_CDS(to_start_list, cutoff_distance_from_start_to_call_spanning=0) -> float:
    spanning_list = [int(read_to_start >= cutoff_distance_from_start_to_call_spanning) for read_to_start in to_start_list]
    number_of_spanning = sum(spanning_list)
    total_number = len(to_start_list)
    fraction_spanning = number_of_spanning / total_number
    return fraction_spanning
span_distance = 0
tqdm.pandas(desc=f"Calculating fraction of reads/transcript that span at least {span_distance}nt from start codons")
compressed_df_transcripts['fraction_spanning'] = compressed_df_transcripts.progress_apply(lambda row: list_to_fraction_spanning_CDS(row['start_distances']), axis=1)
compressed_df_transcripts

Calculating fraction of reads/transcript that span at least 0nt from start codons: 100%|██████████| 43829/43829 [00:00<00:00, 133516.37it/s]


,lib,chr_id,gene_id,gene_name,transcript_id,t5,transcript_hits,mean_polya_length,median_polya_length,mean_read_length,median_read_length,stop_distances,start_distances,transcript_rpm,transcript_frac_hits,fraction_spanning
0,xrn-1-5tera,I,WBGene00000001,aap-1,Y110A7A.10.1,-,16,53.61625,47.095,1389.625,1448.5,"[-1576, -1570, -1568, -1568, -1568, -1568, -13...","[-11, -5, -3, -3, -3, -3, 223, 244, 438, 459, ...",20.408840,0.000020,0.625
1,xrn-1-5tera,I,WBGene00000001,aap-1,Y110A7A.10.1,+,2,21.63000,21.630,1793.000,1793.0,"[-1518, -648]","[47, 917]",2.551105,0.000003,1.000
2,xrn-1-5tera,I,WBGene00000001,aap-1,Y110A7A.10.2,-,10,51.35600,47.095,1164.300,1110.0,"[-1342, -1321, -1127, -1106, -930, -860, -843,...","[223, 244, 438, 459, 635, 705, 722, 727, 868, ...",12.755525,0.000013,1.000
3,xrn-1-5tera,I,WBGene00000001,aap-1,Y110A7A.10.2,+,2,21.63000,21.630,1793.000,1793.0,"[-1518, -648]","[47, 917]",2.551105,0.000003,1.000
4,xrn-1-5tera,I,WBGene00000006,aat-5,C55C2.5a.1,-,2,65.87000,65.870,502.000,502.0,"[-71, -369]","[1539, 1241]",2.551105,0.000003,1.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43824,xrn-1-5tera-smg-6,X,WBGene00206484,C31H2.14,C31H2.14.1,-,1,118.62000,118.620,340.000,340.0,[-227],[-9],8.710194,0.000009,0.000
43825,xrn-1-5tera-smg-6,X,WBGene00206484,C31H2.14,C31H2.14.2,-,4,59.22750,55.325,336.000,336.0,"[-241, -241, -241, -227]","[-23, -23, -23, -9]",34.840778,0.000035,0.000
43826,xrn-1-5tera-smg-6,X,WBGene00219376,C28G1.10,C28G1.10.1,-,4,86.59750,71.680,540.250,557.0,"[-646, -613, -363, -145]","[-38, -5, 245, 463]",34.840778,0.000035,0.500
43827,xrn-1-5tera-smg-6,X,WBGene00219376,C28G1.10,C28G1.10.1,+,1,30.04000,30.040,889.000,889.0,[-734],[-126],8.710194,0.000009,0.000


### Plot *fraction spanning CDS* against *number of adapted reads*
This is the one Josh talked about during out meeting last week (4/22/22)

Some notes:
1. I think this could be better off if we used some kind of "fraction adapted" rather than raw adapted counts. This would likely hold different information, but worth trying!
2. Because of the low adapted read counts, there is a lot of this plot that is basically just showing simple fractions.
3. Could there be a better way to show the ***fraction spanning CDS*** information? Potentially by using the average distance from the start of 5' ends?? Additionally this could be shown as a fraction of the total CDS length so we don't bias ourself into looking at really long transcripts w/ a relativly small portion of the CDS read into.

In [8]:
import plotly.io as pio
pio.renderers.default = "browser"
plot_df = compressed_df_transcripts.query("lib == 'xrn-1-5tera'").query("t5 == '+'")
plot_df['is_MtDNA'] = plot_df['chr_id'] == 'MtDNA'
fig = px.scatter(plot_df,
                 x="transcript_hits",
                 y="fraction_spanning",
                 log_x=True,
                 hover_name="gene_name",
                 hover_data=["gene_id",
                             "transcript_id"],
                 symbol="is_MtDNA",
                 symbol_sequence=[0, 3],)
fig.update_layout(template='plotly_white')
fig.update_traces(marker=dict(size=12,
                              color='DarkSlateGray',
                              opacity=0.5))
fig.write_html(f"/home/marcus/Documents/{get_dt(for_file=True)}_adaptedTranscriptHits-vs-fractionSpanningCDS.html")
fig.show()

## Chi Squared Testing

Continued from the 4th code block

In [144]:
reads_df_genes_raw, compressed_df_genes_raw = load_and_merge_lib_parquets(["xrn-1-5tera", "xrn-1-5tera-smg-6"],
                                                              drop_sub_n=1, add_tail_groupings=False,
                                                                          drop_failed_polya=False,
                                                              group_by_t5=True)
compressed_df_genes_short = compressed_df_genes_raw.copy()[["lib", "chr_id", "gene_id", "gene_name", "t5", "gene_hits", "gene_rpm"]]
compressed_df_genes_short.query("gene_name == 'rpl-12'")

Loading readAssignments file from: /data16/marcus/genomes/elegansRelease100/Caenorhabditis_elegans.WBcel235.100.allChrs.parquet... Done.
Looking for files for libraries: ['xrn-1-5tera', 'xrn-1-5tera-smg-6']
Looking for file for xrn-1-5tera, at /data16/marcus/working/211118_nanoporeRun_totalRNA_5108_xrn-1-KD_5TERA/output_dir/merge_files/*_mergedOnReads.parquet... File Found.
Looking for file for xrn-1-5tera-smg-6, at /data16/marcus/working/211210_nanoporeRun_totalRNA_2102_xrn-1-KD_5TERA/output_dir/merge_files/*_mergedOnReads.parquet... File Found.
Loading parquet for xrn-1-5tera lib... Done.
'original_chr_pos' column already found in dataframe, skipping adjustment for 5'ends!
Loading parquet for xrn-1-5tera-smg-6 lib... Done.
'original_chr_pos' column already found in dataframe, skipping adjustment for 5'ends!
Finished assignment merge!. . .

Read counts pre-failed-polyA call drop:   937835
Read counts post-failed-polyA call drop:  607422
Read counts post gene assignment:  607422
Read c

Counting reads per gene: 100%|██████████| 22466/22466 [00:00<00:00, 66438.95it/s]


Gene counts pre sub-1 gene_hits drop:  22466
Gene counts post sub-1 gene_hits drop:  22466


,lib,chr_id,gene_id,gene_name,t5,gene_hits,gene_rpm
9165,xrn-1-5tera,IV,WBGene00004424,rpl-12,-,1412,3984.940762
9166,xrn-1-5tera,IV,WBGene00004424,rpl-12,+,37,104.421252


In [127]:
conversion_dict = {"xrn-1-5tera": "n2",
                   "xrn-1-5tera-smg-6": "s6"}
ans = [y for x, y in compressed_df_genes_short.groupby(['lib', 't5'], as_index=False)]
df_dict = {}
for i, df in enumerate(ans):
    lib = df.lib.unique()[0]
    t5 = df.t5.unique()[0]
    df = df[["chr_id", "gene_id", "gene_name", "gene_hits", "gene_rpm"]]
    df = df.rename(columns={col: f'{col}_{conversion_dict[lib]}_t5{t5}' for col in df.columns if col not in ["chr_id", "gene_id", "gene_name"]})
    df_dict[(conversion_dict[lib], t5)] = df.set_index(["chr_id", "gene_id", "gene_name"])
    print((conversion_dict[lib], t5))
    print(df_dict[(conversion_dict[lib], t5)].query("gene_name == 'rpl-12'"))

('n2', '+')
                                 gene_hits_n2_t5+  gene_rpm_n2_t5+
chr_id gene_id        gene_name                                   
IV     WBGene00004424 rpl-12                   37       104.421252
('n2', '-')
                                 gene_hits_n2_t5-  gene_rpm_n2_t5-
chr_id gene_id        gene_name                                   
IV     WBGene00004424 rpl-12                 1412      3984.940762
('s6', '+')
Empty DataFrame
Columns: [gene_hits_s6_t5+, gene_rpm_s6_t5+]
Index: []
('s6', '-')
Empty DataFrame
Columns: [gene_hits_s6_t5-, gene_rpm_s6_t5-]
Index: []


In [67]:
super_df = pd.concat(df_dict.values(), axis=1, join='outer').fillna(0)
super_df

gene_hits_n2_t5+  gene_rpm_n2_t5+  gene_hits_n2_t5-  gene_rpm_n2_t5-  gene_hits_s6_t5+  gene_rpm_s6_t5+  gene_hits_s6_t5-  gene_rpm_s6_t5-
chr_id gene_id        gene_name                                                                                                                                            
I      WBGene00000001 aap-1                   2.0         5.644392              16.0        45.155136               1.0        16.804463               0.0          0.00000
       WBGene00000006 aat-5                   1.0         2.822196               2.0         5.644392               0.0         0.000000               0.0          0.00000
       WBGene00000010 aat-9                   0.0         0.000000               5.0        14.110980               0.0         0.000000               0.0          0.00000
       WBGene00000020 abt-2                   0.0         0.000000               3.0         8.466588               0.0         0.000000               0.0          0.00000
       WBGene00000023 abt-5                   1.0         2.822196               1.0         2.822196               0.0         0.000000               0.0          0.00000
...                                           ...              ...               ...              ...               ...              ...               ...              ...
X      WBGene00269392 F54F7.14                0.0         0.000000               1.0         2.822196               0.0         0.000000               0.0          0.00000
       WBGene00271606 F22F4.9                 0.0         0.000000               1.0         2.822196               0.0         0.000000               0.0          0.00000
       WBGene00271701 F10D7.10                1.0         2.822196               0.0         0.000000               0.0         0.000000               0.0          0.00000
       WBGene00284849 C03A3.9                 0.0         0.000000               1.0         2.822196               0.0         0.000000               3.0         50.41339
       WBGene00303024 H40L08.7                6.0        16.933176               6.0        16.933176               0.0         0.000000               0.0          0.00000

[12591 rows x 8 columns]

In [1]:
def make_contingency_table(row):
    array = np.array([[row["gene_hits_n2_t5-"][0], row["gene_hits_s6_t5-"][0]],
                      [row["gene_hits_n2_t5+"][0], row["gene_hits_s6_t5+"][0]]])
    little_df = pd.DataFrame(array, columns=['n2', 'smg-6'], index=['t5 -', 't5 +'])
    return little_df

make_contingency_table(super_df.query("gene_name == 'aco-1'"))

NameError: name 'make_contingency_table' is not defined

In [132]:
from scipy.stats import chi2_contingency, chisquare
def test_gene_chi2(gene):
    example_case = make_contingency_table(super_df.query(f"gene_name == '{gene}'"))
    print(f"{gene}:")
    print(example_case)
    chi2, p, deg_of_free, expected = chi2_contingency(np.array(example_case))
    print(f"{'chi2':>20} = {chi2:.6f}\n"
          f"{'p value':>20} = {p:.3g}\n"
          f"{'degrees of freedom':>20} = {deg_of_free:.6f}\n"
          #f"{'expected:':>20}\n{expected}"
          )

# Example case:
example_genes = ['acdh-1', 'xrn-1', 'nduo-3', 'rpl-22', 'ets-4', 'ubl-1', 'rpl-12'
                                                                          ]
for example_gene in example_genes:
    test_gene_chi2(example_gene)


acdh-1:
         n2  smg-6
t5 +  245.0  106.0
t5 -   16.0    9.0
                chi2 = 0.147100
             p value = 0.701
  degrees of freedom = 1.000000

xrn-1:
          n2   smg-6
t5 +  3107.0  1042.0
t5 -   142.0     9.0
                chi2 = 27.916579
             p value = 1.27e-07
  degrees of freedom = 1.000000

nduo-3:
          n2   smg-6
t5 +  1129.0  1493.0
t5 -   658.0  1355.0
                chi2 = 51.266635
             p value = 8.06e-13
  degrees of freedom = 1.000000

rpl-22:
         n2  smg-6
t5 +  550.0  231.0
t5 -    8.0    4.0
                chi2 = 0.001278
             p value = 0.971
  degrees of freedom = 1.000000

ets-4:
        n2  smg-6
t5 +  13.0    6.0
t5 -   3.0    1.0
                chi2 = 0.114162
             p value = 0.735
  degrees of freedom = 1.000000

ubl-1:
          n2  smg-6
t5 +  1457.0  426.0
t5 -    46.0    1.0
                chi2 = 10.022249
             p value = 0.00155
  degrees of freedom = 1.000000



In [2]:
# Lets try w/ another dimension (being genes)
gene_array_3d = np.array([np.array(make_contingency_table(super_df.query(f"gene_name == '{gene}'"))) for gene in example_genes])

chi2_contingency(gene_array_3d)

NameError: name 'pd' is not defined

In [ ]:
pd.read_parquet("/data16/marcus/working/211210_nanoporeRun_totalRNA_2102_xrn-1-KD_5TERA/output_dir/merge_files/211214_mergedOnReads.parquet")